In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
cleandf = pd.read_csv('cleaned.csv')
cleandf.head()

,Unnamed: 0,id,prices_amountMax,prices_amountMin,prices_availability,prices_condition,prices_currency,prices_dateSeen,prices_isSale,prices_merchant,...,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight
0,0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,...,NaN,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,NaN,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.93796E+11,32.8 pounds
1,1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,...,NaN,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds
2,2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,...,NaN,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds
3,3,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,...,NaN,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds
4,4,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,...,NaN,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds


In [3]:
# delete non-USD currencies
# rename price_conditions columns so blank = 'new' and all others = 'used'
# Lump all merchants into either Walmart, amazon, Best Buy or 'other'
# break categories column into its separate components

In [4]:
cleandf.shape

(14592, 27)

In [5]:
cleandf = cleandf[cleandf["prices_currency"].str.contains("CAD|EUR|GBP|SGD")==False]
cleandf.shape

(14496, 27)

In [6]:
import re
cleandf["prices_condition"].fillna("New", inplace= True)

In [7]:
#cleandf2 = cleandf["prices_condition"].str.replace(r'new|New\sother\s.(see\sdetails.)/i', 'New', regex = True)
cleandf2 = cleandf
cleandf["prices_condition"].value_counts()

New                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [8]:
cleandf2.loc[cleandf2['prices_condition'].str.contains(r'[Rr]efurbished'), 'prices_condition'] = 'Used'
cleandf2["prices_condition"].value_counts()

New                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [9]:
cleandf2.loc[cleandf2['prices_condition'].str.contains(r'[nN]ew'), 'prices_condition'] = 'New'
cleandf2["prices_condition"].value_counts()

New                         13806
Used                          678
pre-owned                       7
For parts or not working        5
Name: prices_condition, dtype: int64

In [10]:
cleandf2.loc[cleandf2['prices_condition'].str.contains('pre-owned'), 'prices_condition'] = 'Used'
cleandf2["prices_condition"].value_counts()

New                         13806
Used                          685
For parts or not working        5
Name: prices_condition, dtype: int64

In [11]:
cleandf2 = cleandf2[cleandf2['prices_condition'].str.contains('parts') == False]
cleandf2["prices_condition"].value_counts()

New     13806
Used      685
Name: prices_condition, dtype: int64

In [12]:
mask = (cleandf2.prices_merchant != None) & (cleandf2.prices_merchant.str.contains(r'^[Bb]est'))
column_name = 'prices_merchant'
cleandf2.loc[mask, column_name] = 'Bestbuy.com'

C:\Users\TBrecke_PC\anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
mask2 = (cleandf2.prices_merchant != None) & (cleandf2.prices_merchant.str.contains(r'[Ww]almart'))
cleandf2.loc[mask2, column_name] = 'Walmart.com'

mask3 = (cleandf2.prices_merchant != None) & (cleandf2.prices_merchant.str.contains(r'[Aa]mazon'))
cleandf2.loc[mask3, column_name] = 'Amazon.com'

mask4 = (cleandf2.prices_merchant != None) & (cleandf2.prices_merchant.str.contains(r'[Bb]hphoto'))
cleandf2.loc[mask4, column_name] = 'bhphotovideo.com'

mask5 = (cleandf2.prices_merchant != 'Bestbuy.com') & (cleandf2.prices_merchant != 'Walmart.com') & (cleandf2.prices_merchant != 'Amazon.com') & (cleandf.prices_merchant != 'bhphotovideo.com')
cleandf2.loc[mask5, column_name] = 'Other'

cleandf2.prices_merchant.value_counts().head(50)

Other               5405
Bestbuy.com         4551
bhphotovideo.com    2744
Walmart.com         1395
Amazon.com           396
Name: prices_merchant, dtype: int64

In [14]:
# Old stuff below

In [15]:
# test = (cleandf2.prices_condition != None) & (cleandf2.prices_condition.str.contains(r'[Rr]efurbished'))
# column = 'prices_condition'
# cleandf2.loc[test, column] = 'Used'
# cleandf2["prices_condition"].value_counts()

In [16]:
# cleandf2["prices_condition"] = cleandf2["prices_condition"].str.replace(r'\b[Rr]efurbished\b', 'Used', regex = True)
# # cleandf2.str.replace(r'\b[Uu]sed\b', 'Used', regex = True)
# cleandf2["prices_condition"].value_counts()

In [17]:
# cleandf2["prices_condition"] = cleandf2["prices_condition"].str.replace(r'\*\b[Uu]sed\b', 'Used', regex = True)
# cleandf2["prices_condition"].value_counts()

In [18]:
# import re
# mask = (cleandf.prices_merchant != None) & (cleandf.prices_merchant.str.contains(r'^[Bb]est'))
# column_name = 'prices_merchant'
# cleandf.loc[mask, column_name] = 'Bestbuy.com'

In [19]:
# mask2 = (cleandf.prices_merchant != None) & (cleandf.prices_merchant.str.contains(r'[Ww]almart'))
# cleandf.loc[mask2, column_name] = 'Walmart.com'

# mask3 = (cleandf.prices_merchant != None) & (cleandf.prices_merchant.str.contains(r'[Aa]mazon'))
# cleandf.loc[mask3, column_name] = 'Amazon.com'

# mask4 = (cleandf.prices_merchant != None) & (cleandf.prices_merchant.str.contains(r'[Bb]hphoto'))
# cleandf.loc[mask4, column_name] = 'bhphotovideo.com'

# mask5 = (cleandf.prices_merchant != 'Bestbuy.com') & (cleandf.prices_merchant != 'Walmart.com') & (cleandf.prices_merchant != 'Amazon.com') & (cleandf.prices_merchant != 'bhphotovideo.com')
# cleandf.loc[mask5, column_name] = 'Other'

# cleandf.prices_merchant.value_counts().head(50)

In [20]:
#old attempts below

In [21]:
# #example of how to change condition and merchant
# substring = "Boytone"
# name = cleandf["name"]

# for i in name:
#     if substring in i:
#         print("substring in string")
#         #cleandf[i] = substring
#     else:
#         print("N/A")
    

In [22]:
# cleandf["prices_condition"].fillna("New", inplace= True)

In [23]:
# cleandf = cleandf.replace({'Refurbished': 'Used', 'Seller refurbished': 'Used',
#                                'pre-owned': 'Used',"Manufacturer refurbished": "Used" })
# #condition.to_csv('item_condition.csv')

In [24]:
# cleandf["prices_condition"]

In [25]:
# cleandf["prices_condition"].replace("new", "New")
# cleandf["prices_condition"].value_counts()

In [26]:
# substring = "Boytone"
# name = cleandf["prices_condition"]

# for i in name:
#     if "Brand New" in i:
#         name.replace(i, "New")
#         #cleandf[i] = substring
#     elif "New other (see details)" in i:
#         name.replace(i, "New")
#     elif "Seller refurbished" in i:
#         name.replace(i, "Used")
#     elif "Refurbished" in i:
#         name.replace(i, "Used")
#     elif "pre-owned" in i:
#         name.replace(i, "Used")
#     elif "Manufacturer refurbished" in i:
#         name.replace(i, "Used")


In [27]:
# name.value_counts()

In [28]:
# cleandf["prices_condition"].value_counts()